In [1]:
# General Tools
import numpy as np
import scipy as sp
import pandas as pd
from datetime import time, datetime

# Misc
import random

import os
import datetime
from platform import python_version

# EDA Tools
import ppscore as pps #<! See https://github.com/8080labs/ppscore -> pip install git+https://github.com/8080labs/ppscore.git

# Ensemble Engines
import lightgbm
import xgboost

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show

# Jupyter
from ipywidgets import interact, Dropdown, Layout

In [2]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

sns.set_theme() #>! Apply SeaBorn theme

In [3]:
# Constants
DATA_FOLDER_NAME    = 'DataSet'
#DATA_FILE_EXT       = 'csv'
# Parameters
csvFileName ='AnibusDAO001.csv'#'Dataset Bit2C.csv'#'Dataset Bit2C Test Case.csv' #
# Loading / Generating Data
#dfData = pd.read_csv(os.path.join(DATA_FOLDER_NAME, csvFileName))
#numRows, numCols = dfData.shape
#print(f'The number of rows (Samples): {numRows}, The number of columns: {numCols}')

In [112]:
class feature_calculations:
    """
    class for calculations of features
    
    Total Value in ETH sent by a wallet
    Total Received value in ETH by a wallet
    Average value in ETH sent by a wallet
    Average value in ETH recieved by a Wallet
    Standard deviation recieved
    Standard deviation sent
    Account first transaction
    Account last transaction
    A number of transaction sent by a wallet
    A number of transaction sent by a wallet over time period
    A number of transaction recieved by a wallet
    A number of transaction recieved by a wallet over a time period
    Average time between transaction performed by sending wallet
    Average time between transaction performed by received by wallet
    Standard deviation of time between transaction performed by sending wallet
    Standard deviation of transaction time in seconds receiving wallet


    """
    def __init__(self,
                 folder_name: str,  
                 file_name: str 
                 ):
                                  
        """
        Args:
        """

        self.folder_name = folder_name
        self.file_name = file_name
        self.df = pd.read_csv(os.path.join(folder_name, file_name))
        self.amount_str = 'Amount'
        self.time_str = 'Transaction Time'
        self.time_diff_str = 'Time Difference [Sec]'
        self.currency_type_str  ='Currency Type'
        
    def format_dt(self):
        self.df[self.time_str] = pd.to_datetime(self.df[self.time_str], infer_datetime_format = 'True')

    def total_value_sent_by_wallet(self, currency = None): 
        'Total Value in ETH sent by a wallet'
        
        df = self.df
        if currency:
            val = df.loc[df['Currency'] == currency, self.amount_str].sum()
            self.df['Total Value in {} sent by a wallet'.format(currency)] = val 
        else:
            val = df[self.amount_str].sum()    
            self.df['Total Value sent by a wallet'] = val

    def total_value_received_by_wallet(self , currency = None):
        'Total Value in ETH sent by a wallet'
        
        df = self.df
        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('sum').set_index('Receiver ID').to_dict()['Amount']
            self.df['Total Value in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('sum').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Total Value received by a wallet'] = df.apply(lambda x: dct[x['Receiver ID']], axis=1)#df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 )

    def average_value_sent_by_wallet(self, currency = None): 
        'Average value in ETH sent by a wallet'
        
        df = self.df
        if currency:
            val = df.loc[df['Currency'] == currency, self.amount_str].mean()
            self.df['Average Value in {} sent by a wallet'.format(currency)] = val 
        else:
            val = df[self.amount_str].mean()    
            self.df['Average Value sent by a wallet'] = val  

    def average_value_received_by_wallet(self , currency = None):
        'Average value in ETH recieved by a Wallet'
        df = self.df
        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('mean').set_index('Receiver ID').to_dict()['Amount']
            self.df['Average Value in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('mean').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Average Value received by a wallet'] = df.apply(lambda x: dct[x['Receiver ID']], axis=1)#df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 )

    def std_value_sent_by_wallet(self, currency = None): 
        'Standard deviation sent'
        
        df = self.df
        if currency:
            val = df.loc[df['Currency'] == currency, self.amount_str].std()
            self.df['STD Value in {} sent by a wallet'.format(currency)] = val 
        else:
            val = df[self.amount_str].std()    
            self.df['STD Value sent by a wallet'] = val


    def std_value_received_by_wallet(self , currency = None):
        'Standard deviation recieved'

        df = self.df
        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('std').set_index('Receiver ID').to_dict()['Amount']
            self.df['STD Value in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('std').set_index('Receiver ID').to_dict()['Amount']    
            self.df['STD Value received by a wallet'] = df.apply(lambda x: dct[x['Receiver ID']], axis=1)#df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 )
    
    def number_of_trans_sent_by_wallet(self, currency = None):
        'A number of transaction sent by a wallet'
        
        df = self.df
        if currency:
            val = df.loc[df['Currency'] == currency, self.amount_str].count()
            self.df['Number of transactions in {} sent by a wallet'.format(currency)] = val 
        else:
            val = df[self.amount_str].count()    
            self.df['Number of transactions sent by a wallet'] = val
              
    
    def number_of_trans_received_by_wallet(self , currency = None):
        'A number of transaction recieved'

        df = self.df
        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('count').set_index('Receiver ID').to_dict()['Amount']
            self.df['Number of transactions in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('count').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Number of transactions received by a wallet'] = df.apply(lambda x: dct[x['Receiver ID']], axis=1)#df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 )
    
    def number_of_trans_sent_by_wallet_over_time(self, currency = None , start_date = None ,  end_date = None): 
        'A number of transaction sent by a wallet over time period'
        
        df = self.df
        ###### TIME COMPARISON
        #df = (df[self.time_str].dt.datetime > start_date) & (df[self.time_str].dt.datetime <= end_date)   pd.to_datetime(datetime.datetime(2010,10,10 , 21, 26, 49, 135154)).tz_localize('UTC')
        start_date = pd.to_datetime(start_date).tz_localize('UTC'); end_date = pd.to_datetime(end_date).tz_localize('UTC')
        df = df[df[self.time_str].between(start_date, end_date)]
        if currency:
            val = df.loc[df['Currency'] == currency, self.amount_str].count()
            self.df['Number of transactions in {} sent by a wallet over time'.format(currency)] = val 
        else:
            val = df[self.amount_str].count()    
            self.df['Number of transactions sent by a wallet over time'] = val

    def number_of_trans_received_by_wallet_over_time(self , currency = None, start_date = None ,  end_date = None):
        'A number of transaction recieved by wallets over time'

        df = self.df
        start_date = pd.to_datetime(start_date).tz_localize('UTC'); end_date = pd.to_datetime(end_date).tz_localize('UTC')
        df = df[df[self.time_str].between(start_date, end_date)]
        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('count').set_index('Receiver ID').to_dict()['Amount']
            self.df['Number of transactions in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('count').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Number of transactions received by a wallet over time period'] = df.apply(lambda x: dct[x['Receiver ID']], axis=1)#df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 )        

    def average_time_btw_trans_sent_by_wallet(self, currency = None): 
        'Average time between transaction performed by sending wallet'
        
        df = self.df
        df.sort_values(self.time_str, inplace = True)
        df[self.time_diff_str] = df[self.time_str].diff().dt.total_seconds()#.astype(int) #36.225329
        df.loc[0, self.time_diff_str] = 0

        if currency:
            val = df.loc[df['Currency'] == currency, self.time_diff_str].mean()
            self.df['Average time between transaction in {} sent by a wallet'.format(currency)] = val

        else:
            val = df[self.time_diff_str].mean()
            self.df['Average time between transaction sent by a wallet'] = val

    def average_time_btw_trans_received_by_wallet(self, currency = None): ### <-- redo time diff !!!!!!!
        'Average time between transaction received by wallet'
        #################REDO !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! diff not right
        df = self.df
        #df.sort_values(self.time_str, inplace = True)
        dfgru = df.sort_values('Transaction Time').groupby('Receiver ID')
        #for grpn, dfgr in dfgru:
        #    vIndx = df.index
        #    df.loc[vIndx, 'Time Difference Group [Sec]'] = dfgr['Transaction Time'].diff().dt.total_seconds()
        #    df.loc[vIndx[0], 'Time Difference Group [Sec]'] = 0

        df['Time Difference Group [Sec]'] = df.sort_values([self.time_str]).groupby('Receiver ID')[self.time_str].diff().dt.total_seconds()#.astype(int)

        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Time Difference Group [Sec]'].agg('mean').set_index('Receiver ID').to_dict()['Time Difference Group [Sec]']
            self.df['Average time between transaction in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.groupby(['Receiver ID'], as_index=False)['Time Difference Group [Sec]'].agg('mean').set_index('Receiver ID').to_dict()['Time Difference Group [Sec]']    
            self.df['Average time between transaction received by a wallet'] = df.apply(lambda x: dct[x['Receiver ID']], axis=1)

    def std_time_btw_trans_sent_by_wallet(self, currency = None): 
        'Standard deviation of time between transaction performed by sending wallet'
        
        df = self.df
        df.sort_values(self.time_str, inplace = True)
        df[self.time_diff_str] = df[self.time_str].diff().dt.total_seconds()
        df.loc[0, self.time_diff_str] = 0
        
        if currency:
            val = df.loc[df['Currency'] == currency, self.time_diff_str].std()
            self.df['STD time between transaction in {} sent by a wallet'.format(currency)] = val

        else:
            val = df[self.time_diff_str].std()
            self.df['STD time between transaction sent by a wallet'] = val
        
    def time_diff_betwenn_first_and_most_sent(self, currency = None): ##############redo currency!!!!!!!!!!!!!!!!!!!!!
        'Time difference between sent first and most'
        df = self.df

        if currency:                            
            first_time_stp = df.loc[df['Currency'] == currency, self.time_str].min() 
            max_val =df.loc[df['Currency'] == currency, self.amount_str].max() #df[self.amount_str].max()
            #time_stp_of_max_val = df.loc[df[self.amount_str] == max_val and df['Currency'] == currency][self.time_str]
            df_cur = df.loc[df['Currency'] == currency]
            time_stp_of_max_val = df_cur.loc[df_cur[self.amount_str] == max_val][self.time_str]
            if time_stp_of_max_val.shape[0] > 1:
                time_stp_of_max_val = time_stp_of_max_val.iloc[0]  
            #time_stp_of_max_val = df.loc[(df[self.amount_str] == max_val) & (df['Currency'] == currency)][self.time_str]
            self.df['Time difference between sent first and most in {}'.format(currency)] =  int((time_stp_of_max_val - first_time_stp).dt.total_seconds())

        else:
            
            first_time_stp = df[self.time_str].min() 
            max_val = df[self.amount_str].max()
            time_stp_of_max_val = df.loc[df[self.amount_str] == max_val][self.time_str]
            #print(time_stp_of_max_val.shape)
            self.df['Time difference between sent first and most'] =  int((time_stp_of_max_val - first_time_stp).dt.total_seconds())

    def time_diff_betwenn_first_and_most_received(self):
        'Time difference between received first and most'    
        df = self.df
        first_time_stp = fc.df['Transaction Time'].min() 
        max_val = fc.df[self.amount_str].max()
        time_stp_of_max_val = df.loc[df[self.amount_str] == max_val][self.time_str]
        self.df['Time difference between received first and most'] =  int((time_stp_of_max_val - first_time_stp).dt.total_seconds())
        



        df = self.df
        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('mean').set_index('Receiver ID').to_dict()['Amount']
            self.df['Average Value in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('mean').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Average Value received by a wallet'] = df.apply(lambda x: dct[x['Receiver ID']], axis=1)#df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 )


    def amount_owned_by_user(self):
        'Represents the amount of ETH owned by the user'
        pass
    
    def amount_of_0_transactions(self, currency = None):
        'Transactions with no amount'
        df = self.df
        if currency:
            val = df.loc[(df['Currency'] == currency) & (df[self.amount_str]) == 0][self.amount_str].count()
            self.df['Transactions in {} with 0 amount'.format(currency)] = val 
        else:
            val = df[df[self.amount_str]==0][self.amount_str].count()
            self.df['Transactions with 0 amount'] = val

    def time_diff_btwn_first_last(self,currency = None):
        'Time difference between the first and last transaction'
        df = self.df
        if currency:
            val_start = df[(df['Currency'] == currency)][self.time_str].min()
            val_end = df[(df['Currency'] == currency)][self.time_str].max()
            val = (val_end - val_start).dt.total_seconds()
            self.df['Time difference between the first and last transaction in {}'.format(currency)] = val
        else:
            val_start = df[self.time_str].min()
            val_end = df[self.time_str].max()
            val = ((val_end - val_start)/np.timedelta64(1, 's'))#(val_end - val_end).dt.total_seconds()
            self.df['Time difference between the first and last transaction'] = val



    def min_max_value_sent_by_wallet(self, currency = None): 
        'Min and Max values sent by a wallet'
        
        df = self.df
        if currency:
            val_min = df.loc[df['Currency'] == currency, self.amount_str].min()
            val_max = df.loc[df['Currency'] == currency, self.amount_str].max()
            self.df['Min Value in {} sent by a wallet'.format(currency)] = val_min
            self.df['Max Value in {} sent by a wallet'.format(currency)] = val_max 
        else:
            val_min = df[self.amount_str].min()
            val_max = df[self.amount_str].max()    
            self.df['Min Value sent by a wallet'] = val_min
            self.df['Max Value sent by a wallet'] = val_max  

    def min_max_value_received_by_wallet(self , currency = None):
        'Min and Max values recieved by a Wallet'
        df = self.df
        if currency:
            dct_min = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('min').set_index('Receiver ID').to_dict()['Amount']
            dct_max = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('max').set_index('Receiver ID').to_dict()['Amount']
            self.df['Min Value in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct_min[x['Receiver ID']] if x['Receiver ID'] in dct_min else 0 , axis=1 )
            self.df['Max Value in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct_max[x['Receiver ID']] if x['Receiver ID'] in dct_max else 0 , axis=1 )   
        else:
            dct_min = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('min').set_index('Receiver ID').to_dict()['Amount']
            dct_max = df.groupby(['Receiver ID'], as_index=False)['Amount'].agg('max').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Min Value received by a wallet'] = df.apply(lambda x: dct_min[x['Receiver ID']], axis=1)
            self.df['Max Value received by a wallet'] = df.apply(lambda x: dct_max[x['Receiver ID']], axis=1)


    def number_created_contract_transactions(self):
        'Total Number of created contract transactions  = groupby receiver, count everything NOT in ETH'
        df = self.df

        dct = df.loc[df['Currency'] != 'ETH'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('count').set_index('Receiver ID').to_dict()['Amount']    
        self.df['Total Number of created contract transactions'] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1)


    def min_max_avg_value_of_ether_sent_to_contract(self, option = None):  ##### groupping or not need to be verified !!!!!!!!!!!!!!!!!!!!!!!!!!!
        '''
        Minimum value of Ether sent to a contract	everything NOT Etherium
        Maximum value of Ether sent to a contrac	everything NOT Etherium
        Average value of Ether sent to contracts	everything NOT Etherium
        '''
        df = self.df
        
        if not option:

            dct_min = df.loc[df['Currency'] != 'ETH'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('min').set_index('Receiver ID').to_dict()['Amount']
            dct_max = df.loc[df['Currency'] != 'ETH'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('max').set_index('Receiver ID').to_dict()['Amount']
            dct_avg = df.loc[df['Currency'] != 'ETH'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('mean').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Min value of Ether sent to a contract'] = df.apply(lambda x: dct_min[x['Receiver ID']] if x['Receiver ID'] in dct_min else 0 , axis=1)
            self.df['Max value of Ether sent to a contract'] = df.apply(lambda x: dct_max[x['Receiver ID']] if x['Receiver ID'] in dct_max else 0 , axis=1)
            self.df['Avg value of Ether sent to a contract'] = df.apply(lambda x: dct_avg[x['Receiver ID']] if x['Receiver ID'] in dct_avg else 0 , axis=1)
        else:
            val_min = df.loc[df['Currency'] != 'ETH', self.amount_str].min()
            val_max = df.loc[df['Currency'] != 'ETH', self.amount_str].max()
            val_avg = df.loc[df['Currency'] != 'ETH', self.amount_str].mean()
            self.df['Min value of Ether sent to a contract'] = val_min
            self.df['Max value of Ether sent to a contract'] = val_max    
            self.df['Avg value of Ether sent to a contract'] = val_avg

    def total_value_ERC20_sent_by_wallet(self, currency = None): 
        'Total Value in ETH sent by a wallet'
        
        df = self.df
        if currency:
            val = df.loc[df['Currency'] == currency, self.amount_str].sum()
            self.df['Total Value in {} sent by a wallet'.format(currency)] = val 
        else:
            val = df.loc[df[self.currency_type_str] == 'ERC20', self.amount_str].sum()    
            self.df['Total ERC20 token Value sent '] = val

    def total_number_ERC20_transactions_sent(self, currency = None): 
        'Total number ERC20 token transactions sent'
        
        df = self.df
        if currency:
            val = df.loc[(df['Currency'] == currency) & (df[self.currency_type_str] == 'ERC20'), self.amount_str].sum()
            self.df['Total number in {}  ERC20 tokens transactions sent'.format(currency)] = val 
        else:
            val = df.loc[df[self.currency_type_str] == 'ERC20', self.amount_str].count()    
            self.df['Total number ERC20 token transactions sent'] = val



    def total_value_ERC20_received_by_wallet(self , currency = None):
        'Total Value in ETH sent by a wallet'
        
        df = self.df
        if currency:
            dct = df.loc[df['Currency'] == currency].groupby(['Receiver ID'], as_index=False)['Amount'].agg('sum').set_index('Receiver ID').to_dict()['Amount']
            self.df['Total Value in {} received by a wallet'.format(currency)] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1 )   
        else:
            dct = df.loc[df[self.currency_type_str] == 'ERC20'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('sum').set_index('Receiver ID').to_dict()['Amount']    
            self.df['Total ERC20 Value received by a wallet'] = df.apply(lambda x: dct[x['Receiver ID']] if x['Receiver ID'] in dct else 0 , axis=1)


    def min_max_avg_value_in_ether_from_ERC20_received(self, option = None):  ##### groupping or not need to be verified !!!!!!!!!!!!!!!!!!!!!!!!!!!
        '''
        Minimum value in Ether received from ERC20 token transactions for account
        Maximum value in Ether received from ERC20 token transactions for account
        Average value in Ether received from ERC20 token transactions for account

        '''
        df = self.df
        
        dct_min = df.loc[df[self.currency_type_str] == 'ERC20'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('min').set_index('Receiver ID').to_dict()['Amount']
        dct_max = df.loc[df[self.currency_type_str] == 'ERC20'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('max').set_index('Receiver ID').to_dict()['Amount']
        dct_avg = df.loc[df[self.currency_type_str] == 'ERC20'].groupby(['Receiver ID'], as_index=False)['Amount'].agg('mean').set_index('Receiver ID').to_dict()['Amount']    
        self.df['Min value in Ether received from ERC20 for account'] = df.apply(lambda x: dct_min[x['Receiver ID']] if x['Receiver ID'] in dct_min else 0 , axis=1)
        self.df['Max value in Ether received from ERC20 for account'] = df.apply(lambda x: dct_max[x['Receiver ID']] if x['Receiver ID'] in dct_max else 0 , axis=1)
        self.df['Avg value in Ether received from ERC20 for account'] = df.apply(lambda x: dct_avg[x['Receiver ID']] if x['Receiver ID'] in dct_avg else 0 , axis=1)
    
    
    def min_max_avg_value_in_ether_from_ERC20_sent(self):
        df = self.df 
        val_min = df.loc[df[self.currency_type_str] == 'ERC20', self.amount_str].min()
        val_max = df.loc[df[self.currency_type_str] == 'ERC20', self.amount_str].max()
        val_avg = df.loc[df[self.currency_type_str] == 'ERC20', self.amount_str].mean()
        self.df['Min value of Ether sent from ERC20 for account'] = val_min
        self.df['Max value of Ether sent from ERC20 for account'] = val_max    
        self.df['Avg value of Ether sent from ERC20 for account'] = val_avg
    

In [105]:
fc = feature_calculations(folder_name = DATA_FOLDER_NAME , file_name = csvFileName)
#fc.df
fc.format_dt() 
fc.total_value_sent_by_wallet()#(currency = 'ETH')
fc.total_value_received_by_wallet()#(currency = 'ETH')
fc.average_value_sent_by_wallet()#(currency = 'ETH')
fc.average_value_received_by_wallet(currency = 'ETH')
fc.std_value_sent_by_wallet(currency='ETH')
#fc.std_value_received_by_wallet()#(currency='ETH')
#fc.number_of_trans_sent_by_wallet(currency='ETH')
#fc.number_of_trans_received_by_wallet() ######### sum(fc.df['Number of transactions received by a wallet'].unique())
#Timestamp('2021-10-20 12:21:53+0000', tz='UTC')) (Timestamp('2021-10-30 23:56:38+0000', tz='UTC'),
#fc.number_of_trans_sent_by_wallet_over_time( currency = None , start_date = datetime.datetime(2021, 10, 25, 12, 21, 53, 0) ,  end_date = datetime.datetime(2021, 10, 30, 23, 56, 38, 0)) 
#fc.average_time_btw_trans_sent_by_wallet()
#fc.average_time_btw_trans_received_by_wallet()
#fc.std_time_btw_trans_sent_by_wallet(currency='ETH')
#fc.average_time_btw_trans_sent_by_wallet()
#fc.average_time_btw_trans_received_by_wallet()
#fc.time_diff_betwenn_first_and_most_sent(currency='ETH')
fc.average_time_btw_trans_sent_by_wallet()

In [106]:
fc.df.head(7)
type(fc.df.iloc[0])

pandas.core.series.Series